<a href="https://colab.research.google.com/github/annemiekvdleest/Machine_Learning/blob/main/ML_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- look at the statistics of the data (df. describe or something)
- are the data on the same scale (normalizations)
- think about the model space H discussed in class
- browse a bit for which model would work better for the attributes of the data given
- the bias-variance tradeoff
- how can we select the appropriate features
- how best to deal with the nans. 

It's always a good idea to have a baseline like autoML then you know what scores you have to improve
https://machinelearningmastery.com/automl-libraries-for-python/


The goal of the Assignment is to predict customers engagement based on certain parameters/features.
0 class means the deals has not started, 1 means in progress and 2 means done/closed 

# Become one with the data
- look at distribution and patterns
- data imbalances and biases
- How much variation is there and what form does it take? What variation is spurious and could be preprocessed out?
- Does spatial position matter or do we want to average pool it out? How much does detail matter and how far could we afford to downsample the images? How noisy are the labels?
- search/filter/sort by whatever you can think of (e.g. type of label, size of annotations, number of annotations, etc.) and visualize their distributions and the outliers along any axis

In [ ]:
#import
import pandas as pd

In [ ]:
df = pd.read_csv('train_data.csv')
print(df)

           opportunity_id  current_date_day  current_date_month  \
0       15427464874523186                 2                  11   
1       15427464874523186                 9                  11   
2       53852196172413351                11                  10   
3       63848905825323247                25                   7   
4       63848905825323247                 1                   8   
...                   ...               ...                 ...   
1588  9186677120757296287                22                  11   
1589  9186677120757296287                29                  11   
1590  9186677120757296287                 6                  12   
1591  9188987770029297576                 1                   7   
1592  9207329380392407700                 4                  11   

      current_date_year  TOPICS__mean_calls_Integration__from_opp_creation  \
0                  2021                                                NaN   
1                  2021                

# Set up the end-to-end training/evaluation skeleton + get dumb baselines
- pick some simple model that you couldn’t possibly have screwed up somehow - e.g. a linear classifier, or a very tiny ConvNet
- train it, visualize the losses, any other metrics (e.g. accuracy), model predictions, and perform a series of ablation experiments with explicit hypotheses along the way
- check tips and tricks on website

http://karpathy.github.io/2019/04/25/recipe/

# Overfit + Regularize
- first get a model large enough that it can overfit (i.e. focus on training loss) and then regularize it appropriately (give up some training loss to improve the validation loss)
- check tips and tricks on website

# Tune
- random over grid search
- hyper parameter optimization

# Squeeze out the juice 
- ensembles
- leave it training